In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split

from utils import data_preprocessing_util as dpu
from utils import classification_util as cu

from MLP import MLP
from nn_framework import NNFramework

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
data_path = r'../Ex1/data/Breast_Cancer/breast-cancer-diagnostic.shuf.lrn.csv'
df_original = pd.read_csv(data_path)

random_seed = 32
log_transform = True
outlier_removal = True
scaling = True

scaler = preprocessing.StandardScaler() if scaling else None

df = dpu.preprocess_breast_cancer_data(df_original, log_transform=log_transform, outlier_removal=outlier_removal)

In [5]:
y = df['class']
X = df[df.columns.difference(['ID', 'class'])]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_seed, shuffle=True, stratify=y)


In [6]:
activation_functions = ['relu', 'sigmoid']
learning_rates = [0.0001, 0.001, 0.01, 0.1, 0.5]
hidden_layer_sizes = [(5,), (32,), (16, 16), (10, 5, 5), (16, 8, 8), (64, 32, 32),]

In [7]:
methods = []

for af in activation_functions:
    for lr in learning_rates:
        for hls in hidden_layer_sizes:
            methods.append((f'MLP-{af}-{lr}-{hls}', MLP(n_iter=5000, activation_function=af, learning_rate=lr, hidden_layer_sizes=hls)))
    
pipelines = cu.define_pipelines(methods, scaler=scaler)

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
cv_num = 5
model_params = {}
models = {}
models_lists = {}

for model_name, pipeline in pipelines.items():
    cv_results = cross_validate(pipeline, X, y, cv=cv_num, scoring='f1_macro', return_estimator=True, n_jobs=10)

    models[model_name] = cv_results['estimator']
    model_params[model_name] = {}
    models_lists[model_name] = {}

    num_cols = ['test_score', 'fit_time', 'score_time']

    for num_col in num_cols:
        models_lists[model_name][num_col] = cv_results[num_col]
        model_params[model_name][f'{num_col}_mean'] = cv_results[num_col].mean()
        model_params[model_name][f'{num_col}_std'] = cv_results[num_col].std()
    
    model_params[model_name]['parameter_num'] = cv_results['estimator'][0][model_name].number_of_params_
    model_params[model_name]['hidden_layer_sizes'] = cv_results['estimator'][0][model_name].hidden_layer_sizes
    model_params[model_name]['activation_function'] = cv_results['estimator'][0][model_name].activation_function
    model_params[model_name]['learning_rate'] = cv_results['estimator'][0][model_name].learning_rate
    models_lists[model_name]['converged'] = [e[model_name].converged_ for e in cv_results['estimator']]
    models_lists[model_name]['validation_losses'] = [e[model_name].validation_losses_ for e in cv_results['estimator']]
    models_lists[model_name]['training_losses'] = [e[model_name].training_losses_ for e in cv_results['estimator']]
    model_params[model_name]['num_iter'] = np.array(list([len(e[model_name].training_losses_) for e in cv_results['estimator']])).mean()

    print(model_name)
    print(
        f"f1 scores: {models_lists[model_name]['test_score']}\n" +
        f"f1 mean: {model_params[model_name]['test_score_mean']:.3f}\n" +
        f"f1 std: {model_params[model_name]['test_score_std']:.3f}\n"
    )
    print('----------------------------------------------------------------------------------------------------')
    

Loss did not go down for 10 iterations. Stopping training at iteration 294...
Loss did not go down for 10 iterations. Stopping training at iteration 278...
Loss did not go down for 10 iterations. Stopping training at iteration 264...
Loss did not go down for 10 iterations. Stopping training at iteration 321...
Loss did not go down for 10 iterations. Stopping training at iteration 298...
MLP-relu-0.0001-(5,)
f1 scores: [0.93215739 0.97802198 0.95659722 0.97854998 0.89010989]
f1 mean: 0.947
f1 std: 0.033

----------------------------------------------------------------------------------------------------
Loss did not go down for 10 iterations. Stopping training at iteration 227...
Loss did not go down for 10 iterations. Stopping training at iteration 237...
Loss did not go down for 10 iterations. Stopping training at iteration 246...
Loss did not go down for 10 iterations. Stopping training at iteration 228...
Loss did not go down for 10 iterations. Stopping training at iteration 252...


/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: invalid value encountered in divide
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])


As a punishment we return a model with randomly initialized weights.
MLP-relu-0.0001-(64, 32, 32)
f1 scores: [0.83022071 0.875      0.875      0.87684729 0.47552448]
f1 mean: 0.787
f1 std: 0.156

----------------------------------------------------------------------------------------------------
Loss did not go down for 10 iterations. Stopping training at iteration 81...
Loss did not go down for 10 iterations. Stopping training at iteration 80...
Loss did not go down for 10 iterations. Stopping training at iteration 84...
Loss did not go down for 10 iterations. Stopping training at iteration 85...
Loss did not go down for 10 iterations. Stopping training at iteration 121...
MLP-relu-0.001-(5,)
f1 scores: [0.95404412 1.         1.         0.97854998 0.93406593]
f1 mean: 0.973
f1 std: 0.026

----------------------------------------------------------------------------------------------------
Loss did not go down for 10 iterations. Stopping training at iteration 81...
Loss did not go down 

/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: invalid value encountered in divide
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: invalid value encountered in divide
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: invalid value encountered in divide
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (

Loss did not go down for 10 iterations. Stopping training at iteration 18...
Loss did not go down for 10 iterations. Stopping training at iteration 16...
Loss did not go down for 10 iterations. Stopping training at iteration 23...
Loss did not go down for 10 iterations. Stopping training at iteration 19...
Loss did not go down for 10 iterations. Stopping training at iteration 47...
MLP-relu-0.01-(5,)
f1 scores: [0.95404412 1.         1.         0.97854998 0.95659722]
f1 mean: 0.978
f1 std: 0.020

----------------------------------------------------------------------------------------------------
Loss did not go down for 10 iterations. Stopping training at iteration 18...
Loss did not go down for 10 iterations. Stopping training at iteration 16...
Loss did not go down for 10 iterations. Stopping training at iteration 18...
Loss did not go down for 10 iterations. Stopping training at iteration 19...
Loss did not go down for 10 iterations. Stopping training at iteration 49...
MLP-relu-0.0

...

As a punishment we return a model with randomly initialized weights.

As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
MLP-relu-0.1-(10, 5, 5)
f1 scores: [0.83022071 0.89665151 0.79707792 0.77920514 0.73906062]
f1 mean: 0.808
f1 std: 0.053

----------------------------------------------------------------------------------------------------
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.

As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
MLP-relu-0.1-(16, 8, 8)
f1 scores: [0.39759036 0.3902439  0.3902439  0.3902439  0.390

/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: invalid value encountered in divide
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: invalid value encountered in divide
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: invalid value encountered in divide
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (

As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
MLP-relu-0.5-(16, 8, 8)
f1 scores: [0.39759036 0.3902439  0.3902439  0.3902439  0.3902439 ]
f1 mean: 0.392
f1 std: 0.003

----------------------------------------------------------------------------------------------------
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
MLP-relu-0.5-(64, 32, 32)
f1 scores: [0.45799458 0.3902439  0.3902439  0.3902439  0.47552448]
f1 mean: 0.421
f1 std: 0.038

----------------------------------------------------------------------------------------------------


/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: invalid value encountered in divide
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: invalid value encountered in divide
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: invalid value encountered in divide
  return (

Loss did not go down for 10 iterations. Stopping training at iteration 207...
Loss did not go down for 10 iterations. Stopping training at iteration 187...
Loss did not go down for 10 iterations. Stopping training at iteration 234...
Loss did not go down for 10 iterations. Stopping training at iteration 248...
Loss did not go down for 10 iterations. Stopping training at iteration 285...
MLP-sigmoid-0.0001-(5,)
f1 scores: [0.83637214 0.93406593 0.93406593 1.         0.89010989]
f1 mean: 0.919
f1 std: 0.054

----------------------------------------------------------------------------------------------------
Loss did not go down for 10 iterations. Stopping training at iteration 156...
Loss did not go down for 10 iterations. Stopping training at iteration 178...
Loss did not go down for 10 iterations. Stopping training at iteration 184...
Loss did not go down for 10 iterations. Stopping training at iteration 188...
Loss did not go down for 10 iterations. Stopping training at iteration 220.

In [10]:
df_param = pd.DataFrame(model_params).transpose()
df_param = df_param.reset_index(drop=False)
df_param = df_param.rename(columns={'index': 'model'})

# df_param

In [11]:
df_param.to_csv(r'results/breast_cancer_params_sigmoid.csv', index=False)

In [12]:
df_param.sort_values(['test_score_mean'], ascending=False).head(5)

,model,test_score_mean,test_score_std,fit_time_mean,fit_time_std,score_time_mean,score_time_std,parameter_num,hidden_layer_sizes,activation_function,learning_rate,num_iter
37,"MLP-sigmoid-0.001-(32,)",0.982123,0.017072,2.59008,0.479271,0.009105,0.007221,1058,"(32,)",sigmoid,0.001,65.2
42,"MLP-sigmoid-0.01-(5,)",0.978217,0.013732,0.961008,0.350908,0.008265,0.005729,167,"(5,)",sigmoid,0.01,29.4
12,"MLP-relu-0.01-(5,)",0.977838,0.020001,0.853623,0.326674,0.009437,0.005169,167,"(5,)",relu,0.01,25.6
36,"MLP-sigmoid-0.001-(5,)",0.974018,0.016009,3.3319,0.84472,0.007277,0.004158,167,"(5,)",sigmoid,0.001,101.2
45,"MLP-sigmoid-0.01-(10, 5, 5)",0.974018,0.016009,4.262008,0.437246,0.009032,0.004592,407,"(10, 5, 5)",sigmoid,0.01,68.0


In [14]:
modelsss = df_param.sort_values(['test_score_mean'], ascending=False).head(5)['model']
print(X_train.shape)
results_df = pd.DataFrame()
for mod in modelsss[1:]:
    print(mod)
    pipelines[mod].fit(X_train, y_train)
    y_pred = pipelines[mod].predict(X_test)
    eval_results = cu.eval(y_pred, y_test)
    results_df = results_df.append({
        'Model': mod,
        'cm': eval_results['cm'].flatten(),
        'acc': eval_results['acc'],
        'balanced_acc': eval_results['balanced_acc'],
        'precision': eval_results['precision'],
        'recall': eval_results['recall'],
        'f-score': eval_results['f-score'],
        'support': eval_results['support']
    }, ignore_index=True)

(200, 30)
MLP-sigmoid-0.01-(5,)
Loss did not go down for 10 iterations. Stopping training at iteration 26...
Evaluation metrics:


array([[0.9375    , 0.0625    ],
       [0.05555556, 0.94444444]])

acc: 0.94
balanced_acc: 0.9409722222222222

Macro-averaged precision: 0.9312393887945671
Macro-averaged recall: 0.9409722222222222
Macro-averaged f-score: 0.9356499356499357
Macro-averaged support: None
MLP-relu-0.01-(5,)
Loss did not go down for 10 iterations. Stopping training at iteration 33...
Evaluation metrics:


array([[1.        , 0.        ],
       [0.05555556, 0.94444444]])

acc: 0.98
balanced_acc: 0.9722222222222222

Macro-averaged precision: 0.9848484848484849
Macro-averaged recall: 0.9722222222222222
Macro-averaged f-score: 0.9780219780219781
Macro-averaged support: None
MLP-sigmoid-0.001-(5,)
Loss did not go down for 10 iterations. Stopping training at iteration 111...
Evaluation metrics:


array([[0.9375    , 0.0625    ],
       [0.11111111, 0.88888889]])

acc: 0.92
balanced_acc: 0.9131944444444444

Macro-averaged precision: 0.9131944444444444
Macro-averaged recall: 0.9131944444444444
Macro-averaged f-score: 0.9131944444444444
Macro-averaged support: None
MLP-sigmoid-0.01-(10, 5, 5)
Loss did not go down for 10 iterations. Stopping training at iteration 80...
Evaluation metrics:


array([[0.96875   , 0.03125   ],
       [0.05555556, 0.94444444]])

acc: 0.96
balanced_acc: 0.9565972222222222

Macro-averaged precision: 0.9565972222222222
Macro-averaged recall: 0.9565972222222222
Macro-averaged f-score: 0.9565972222222222
Macro-averaged support: None


In [15]:
results_df.to_csv("results/breast_cancer_trainset.csv", index=False)

In [16]:
results_df

,Model,cm,acc,balanced_acc,precision,recall,f-score,support
0,"MLP-sigmoid-0.01-(5,)","[0.9375, 0.0625, 0.05555555555555555, 0.944444...",0.94,0.940972,0.931239,0.940972,0.935650,None
1,"MLP-relu-0.01-(5,)","[1.0, 0.0, 0.05555555555555555, 0.944444444444...",0.98,0.972222,0.984848,0.972222,0.978022,None
2,"MLP-sigmoid-0.001-(5,)","[0.9375, 0.0625, 0.1111111111111111, 0.8888888...",0.92,0.913194,0.913194,0.913194,0.913194,None
3,"MLP-sigmoid-0.01-(10, 5, 5)","[0.96875, 0.03125, 0.05555555555555555, 0.9444...",0.96,0.956597,0.956597,0.956597,0.956597,None
